# California Collisions Cleaning

## Finding the percentage of collisions in each county caused by drunk-driving incidents

For our first query, from 2010 - 2020 we want to find the total amount of collisions caused by drunk drivers in every county, the total amount of collisions caused by sober drivers in every county and percentage of total collisions caused by drunk drivers in every county


In [ ]:
query = '''
        WITH sober_collisions AS (
        SELECT
          c.county_location,
          COUNT(p.party_sobriety) [Number of Sober Collisions] 
        FROM
          collisions c
        INNER JOIN
          parties p
        ON
          p.case_id = c.case_id
        WHERE
          c.collision_date BETWEEN '2010-01-01' AND '2019-12-31'
        AND
          p.party_sobriety != 'had been drinking, under influence'
        AND
          p.party_type = 'driver'
        AND
          p.at_fault = 1
        GROUP BY
          c.county_location
        )

        SELECT
          d.county_location,
          s.[Number of Sober Collisions],
          d.[Number of Drunk Driving collisions],
          CONCAT(STR(ROUND((d.[Number of Drunk Driving collisions]) * 100.0 /
          (s.[Number of Sober Collisions] + d.[Number of Drunk Driving collisions]), 1)), '%')
          [Percentage drunk driving]
        FROM (
          SELECT
            c.county_location, COUNT(p.party_sobriety) [Number of Drunk Driving collisions]
        FROM
          collisions c
        INNER JOIN
          parties p
        ON
          p.case_id = c.case_id
        WHERE
          c.collision_date BETWEEN '2010-01-01' AND '2019-12-31'
          AND p.party_sobriety = 'had been drinking, under influence'
          AND p.party_type = 'driver'
          AND p.at_fault = 1
        GROUP BY
          c.county_location) d
        INNER JOIN
          sober_collisions s on s.county_location = d.county_location
        '''

In [ ]:
new_data = pd.read_sql_query(query, con)

In [ ]:
new_data

,county_location,Number of Sober Collisions,Number of Drunk Driving collisions,Percentage drunk driving
0,alameda,174322,11564,6.2
1,alpine,595,55,8.5
2,amador,3744,457,10.9
3,butte,15331,2332,13.2
4,calaveras,4424,588,11.7
5,colusa,2650,272,9.3
6,contra costa,79769,7201,8.3
7,del norte,2411,262,9.8
8,el dorado,12505,2105,14.4
9,fresno,58058,6786,10.5


## Collisions By Race

In this query, we want to find the collisions caused by drunk driving for every race.

In [ ]:
query = '''
   SELECT
    c.county_location,
    p.party_race as race_name,
    count(p.party_race) [Number of Drunk Collisions]
   FROM
    collisions c 
   INNER JOIN
    parties p
   ON
    c.case_id = p.case_id
   WHERE
    p.party_type = 'driver'
    AND p.party_sobriety = 'had been drinking, under influence'
    AND c.collision_date BETWEEN '2010-01-01' AND '2019-12-31'
    AND p.at_fault = 1
    AND p.party_race != 'None'
   GROUP BY
    c.county_location, p.party_race
    '''

In [ ]:
race_data = pd.read_sql_query(query, con)

In [ ]:
race_data

,county_location,race_name,Number of Drunk Collisions
0,alameda,asian,751
1,alameda,black,2176
2,alameda,hispanic,4125
3,alameda,other,769
4,alameda,white,3426
...,...,...,...
279,yuba,asian,12
280,yuba,black,21
281,yuba,hispanic,137
282,yuba,other,29


In [ ]:
pd.pivot_table(race_data, values='Number of Drunk Collisions', index=['county_location'],
                    columns=['race_name'], aggfunc=np.sum, fill_value = 0)

race_name,asian,black,hispanic,other,white
county_location,,,,,
alameda,751,2176,4125,769,3426
alpine,0,3,4,4,44
amador,2,7,46,9,386
butte,33,64,356,61,1758
calaveras,3,8,62,7,507
colusa,2,7,128,11,120
contra costa,227,1082,2314,419,2991
del norte,5,2,24,33,194
el dorado,17,28,182,70,1743


## Collisions By age range

For our next query, we want to find the number of collisions in every county for different age groups.

In [ ]:
query = '''
        SELECT
          c.county_location,
          count(p.party_age) [Number of Collisions],
        CASE 
          WHEN p.party_age < 18 THEN 'Under 18'
          WHEN p.party_AGE BETWEEN 18 AND 24 THEN '18-24'
          WHEN p.party_AGE BETWEEN 25 AND 29 THEN '25-29'
          WHEN p.party_AGE BETWEEN 30 AND 34 THEN '30-34'
          WHEN p.party_AGE BETWEEN 35 AND 39 THEN '35-39'
          WHEN p.party_AGE BETWEEN 18 AND 24 THEN '40-44'
          WHEN p.party_AGE BETWEEN 18 AND 24 THEN '45-49'
          WHEN p.party_AGE BETWEEN 18 AND 24 THEN '50-54'
          WHEN p.party_AGE BETWEEN 18 AND 24 THEN '55-59'
          ELSE '60+'
        END AS
          Age_groups
        FROM
          parties p 
        INNER JOIN
          collisions c 
        ON
          c.case_id = p.case_id
        WHERE
          p.party_type = 'driver'
        AND p.party_sobriety = 'had been drinking, under influence'
        AND c.collision_date BETWEEN '2010-01-01' AND '2019-12-31'
        AND p.at_fault = 1
        GROUP BY
          c.county_location, age_groups'''

In [ ]:
age_data = pd.read_sql_query(query, con)

In [ ]:
age_data

,county_location,Number of Collisions,Age_groups
0,alameda,2851,18-24
1,alameda,2337,25-29
2,alameda,1581,30-34
3,alameda,1068,35-39
4,alameda,3588,60+
...,...,...,...
342,yuba,123,25-29
343,yuba,96,30-34
344,yuba,60,35-39
345,yuba,209,60+


In [ ]:
pivot = pd.pivot_table(age_data, values = 'Number of Collisions', index = ['county_location'], 
               aggfunc = np.sum, columns = ['Age_groups'], fill_value = 0)

cols = pivot.columns.tolist()
cols = cols[-1:] + cols[0:-1]
pivot = pivot[cols]
pivot

Age_groups,Under 18,18-24,25-29,30-34,35-39,60+
county_location,,,,,,
alameda,89,2851,2337,1581,1068,3588
alpine,1,11,12,2,2,27
amador,4,87,57,49,32,228
butte,33,726,388,252,171,755
calaveras,4,124,77,56,39,288
colusa,3,83,34,36,23,92
contra costa,99,1782,1375,866,702,2347
del norte,8,55,39,33,31,96
el dorado,27,496,321,244,168,845
